In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals
import random
import json
import os
import time
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
#from sklearn.model_selection import train_test_split
import tensorflow.contrib.legacy_seq2seq as seq2seq
from utilities import show_graph

import unicodedata
import re
import numpy as np
import os
import io
import time
import collections
import json
import string

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [5]:
#def dictionary

In [6]:
def tokenize(line,token='word'):
    if token == 'word':
        return [line.split(' ')]
    elif token == 'char':
        return [list(line)]
    else:
        print('ERROR: unknown token type '+token)

In [7]:
def count_tokens(tokanized_sentences):
    # Flatten a list of token lists into a list of tokens
    tokens = [tk for line in tokanized_sentences for tk in line]
    return collections.Counter(tokens)

In [8]:
filename = 'MLDS_hw2_1_data/training_label.json'
with open(filename, 'r') as f:
    datastore = json.load(f)

In [9]:
vid_feat_set = {}
vid_sentence_set = {}

sizeof_train = 0
for data in datastore:
    video_id = data["id"]
    vid_feat_set[video_id]=None
    vid_sentence_set[video_id]=None
    sizeof_train = sizeof_train+1

In [10]:
batch_size = 50

batches = 1450/batch_size

vid_batch = {}

i = 0
j = 0
for data in datastore:
    vid_feat_list = []
    video_id = data["id"]
    features = np.load("MLDS_hw2_1_data/training_data/feat/{}.npy".format(video_id))
        
    vid_framefeats = []
    for array in features:
        vid_framefeats.append(array)
                
    if j not in vid_batch:
        vid_batch[j] = []

    vid_batch[j].append(vid_framefeats)

    #vid_feat_set[video_id] = vid_framefeats
    #vid_feat_list = np.append(vid_feat_list, vid_framefeats,axis=2)
    i = i+1
     
    if i%batch_size == 0:
        j = j+1

# vid_batches is the video features divided into batches of 50

In [11]:
i = 0
j = 0

sentence_set = {}

for data in datastore:
    
    video_id = data["id"]

    
    #print("reading sentences in: %s" % video_id)
    sentences = data["caption"]
    sentences = [word.lower() for word in sentences] #Normalize the case
    table = str.maketrans('', '', string.punctuation) #Normalize the punctuation
    sentences = [word.translate(table) for word in sentences]
    #print(sentences[0])
    
    sentence_set[i] = sentences[0]
    i = i +1
#Extracting only a single sentence per video

In [12]:
#sentence_set
print("The number of videos in the training set are %d and each video has 80 frames with 4096 features/units each" % len(vid_feat_set))

The number of videos in the training set are 1450 and each video has 80 frames with 4096 features/units each


In [13]:
# print(vid_sentence_set)


In [14]:
# #vid_feat_set['WqQonRVs7WA_0_10.avi']

# count = 0
# for x in vid_sentence_set: 
#     if isinstance(vid_sentence_set[x], list): 
#         count += len(vid_sentence_set[x]) 
# print(count) 

In [15]:
# Mapping string tokens to numertical indices.
def listVocab(sentence_set):
    
    PAD_token = 0
    BOS_token = 1
    EOS_token = 2
    UNK_token = 3
    
    all_tokens = []
    token_index = {"<PAD>": 0,"<BOS>":1,"<EOS>":2,"<UNK>":3}
    index_token = {PAD_token: "<PAD>", BOS_token: "<BOS>", EOS_token: "<EOS>", UNK_token: "<UNK>"}
    
    #for set_i in vid_sentence_set:
    #    sentence_set = vid_sentence_set[set_i]
    #    for line in sentence_set: 
    
    for n in sentence_set:
        line = sentence_set[n]
        tokenized_captions = tokenize(line) #Seperate the words
        all_tokens += tokenized_captions
    
    counter = count_tokens(all_tokens) #Count the word repeatitions in each set
    
    counter_dict = counter.items()
    counter_sort = sorted(counter_dict, key=lambda x:x[1],reverse=True) #sort by frequency of occurance 
    #print(counter_sort)

    i = len(index_token)
    values = [0,1,2,3]
    for token, freq in counter_sort:
        index_token[i] = token
        token_index[token] = i
        values += [i]
        i+=1
    
    return [values,token_index, index_token, len(index_token)]

In [16]:
values,token_index, index_token, nums = listVocab(sentence_set)
print("There are %d unique words in the captions dataset" % nums)

# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)

# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)


There are 1988 unique words in the captions dataset


In [17]:
#counter_dict

In [18]:
#np.shape(onehot_encoded)

In [19]:
def flattenList(nestedList,output): 
    for i in nestedList: 
        if type(i) == list: 
            flattenList(i,output) 
        else: 
            output.append(i) 
            
    return output

In [20]:

def num_encode(test_sentence,index_token,tokenized_sentence=[],num_encoded_sentence=[],onehot_encoded_sentence=[]):
    
    tokenized_sentence.clear()
    num_encoded_sentence.clear()
    #onehot_encoded_sentence.clear()
    
    tokenized_sentence = ["<BOS>"] + tokenize(test_sentence) + ["<EOS>"]
    #print(tokenized_sentence)
    output=[]
    tokenized_sentence = flattenList(tokenized_sentence,output)

    while len(tokenized_sentence) < MAX_WORDS:
        tokenized_sentence.append("<PAD>")
    #print(len(tokenized_sentence))
   
    for token in tokenized_sentence:
        for i in range(0,len(index_token)):
            if token == index_token[i]: 
                num_encoded_sentence.append(i) 
                #onehot_encoded_sentence.append(onehot_encoded[i])
        
    return tokenized_sentence, num_encoded_sentence, onehot_encoded_sentence


In [21]:
MAX_WORDS = 80

tokenizedsentence_batch = {}
intencode_batch = {}
onehot_batch = {}

ii = 0
jj = 0

# for data in datastore:
#     video_id = data["id"]
#     num_encoded_dict[video_id]=None
#     onehot_encoded_dict[video_id]=None
  
    
for n in sentence_set:
    sentence = sentence_set[n]

    tokenized_sentence,encoded_sentence,onehot_encoded_sentence = num_encode(sentence,index_token)
    
    
    encoded_sentence = list(encoded_sentence)
    #onehot_encoded_sentence = list(onehot_encoded_sentence)
    

    #print(type(encoded_sentence)) 
    #print("nxt")
    #num_encoded_per_set.append(encoded_sentence)
    #print(num_encoded_per_set)


    #onehot_encoded_per_set.append(onehot_encoded_sentence)

    #print(num_encoded_per_set)
    
    #num_encoded_dict[vid] = num_encoded_per_set
    #onehot_encoded_dict[vid] = onehot_encoded_per_set
    
    if jj not in intencode_batch:
        #onehot_batch[jj] = []
        intencode_batch[jj] = []
        #tokenizedsentence_batch[jj] = []

    #print(np.shape(onehot_encoded_sentence))    
    #onehot_batch[jj].append(onehot_encoded_sentence)
    intencode_batch[jj].append(encoded_sentence)
    #tokenizedsentence_batch[jj].append(tokenized_sentence)
    
    ii = ii+1
     
    if ii%batch_size == 0:
        jj = jj+1


In [22]:
np.shape(intencode_batch[0])

(50, 80)

In [23]:
len(index_token)

1988

In [24]:
from tensorflow.contrib import rnn

n_hidden = 600
n_words = nums
sizeof_frame = 4096
no_of_frames = 80
sizeof_sentence= 80
learning_rate = 0.01

with tf.Graph().as_default() as graph:

    weights_enc = tf.Variable(tf.random_uniform([sizeof_frame, n_hidden],-0.1,0.1),name="weights_enc")
    bias_enc = tf.Variable(tf.zeros([n_hidden]),name="bias_enc")

    weights_dec = tf.Variable(tf.random_uniform([n_hidden, n_words],-0.1,0.1),name="weights_dec")
    bias_dec = tf.Variable(tf.zeros([n_words]),name="bias_dec")


    x_video = tf.placeholder(tf.float32, (None, no_of_frames, sizeof_frame),'video_features') #inputs
    x_video_flat = tf.reshape(x_video,[-1,sizeof_frame])

    y_label = tf.placeholder(tf.int32,(None, sizeof_sentence),'captions') #outputs

    targets = tf.placeholder(tf.int32,(None,None),'targets') #target (???)

    padding = tf.zeros([batch_size, n_hidden])

    #sampling = tf.placeholder(tf.bool, [sizeof_sentence], name='sampling')
    loss = 0.0

    ########## DATA ###########
    # Example: For i = 0
    #batch_x = np.array(vid_batch[0])
    #batch_y = np.array(intencode_batch[0])
    ###########################

    input_embedding = tf.matmul(x_video_flat,weights_enc) + bias_enc
    input_embedding = tf.reshape(input_embedding,[-1, no_of_frames,n_hidden])
    input_embed = tf.transpose(input_embedding, perm=[1, 0, 2])

    output_embedding = tf.Variable(tf.random_uniform((n_words, n_hidden),-0.1,0.1), name='dec_embedding')
    # output_embed = tf.nn.embedding_lookup(output_embedding,y_label)
    
    ## ENCODING #################################
    
    with tf.variable_scope("LSTM1"):
        lstm1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden,state_is_tuple=True)
    
    with tf.variable_scope("LSTM2"):
        lstm2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden, state_is_tuple=True)

    state1 = lstm1.zero_state(batch_size, dtype=tf.float32)
    state2 = lstm2.zero_state(batch_size, dtype=tf.float32)
    
    for i in range(0, no_of_frames):
        
        if i > 0:
                tf.get_variable_scope().reuse_variables()
                
        with tf.variable_scope("LSTM1"):
            output1, state1 = lstm1(input_embed[i,:,:], state1)

        with tf.variable_scope("LSTM2"):
            output2, state2 = lstm2(tf.concat([padding, output1], axis=1), state2)
    
    ## DECODING ##################################
    
    bos = tf.ones([batch_size, n_hidden])
    padding_in = tf.zeros([batch_size, n_hidden])

    logits = []
    cross_ent_list=[]

    for i in range(0, sizeof_sentence):
        
        tf.get_variable_scope().reuse_variables()

        
        with tf.variable_scope("LSTM1"):
            output1, state1 = lstm1(padding_in, state1)
            
        if i == 0:
            
            with tf.variable_scope("LSTM2"):
                con = tf.concat([bos, output1], axis=1)
                output2, state2 = lstm2(con, state2)
                
        else:
            
            with tf.device("/cpu:0"):
            
                feed_in = y_label[:,i]
                output_embed = tf.nn.embedding_lookup(output_embedding,feed_in)
                

            
            with tf.variable_scope("LSTM2"):
                con = tf.concat([output_embed, output1], axis=1)
                output2, state2 = lstm2(con, state2)

        logit_words = tf.matmul(output2, weights_dec) + bias_dec
        logits.append(logit_words)

        word_i = y_label[:,i]

        one_hot_labels = tf.one_hot(word_i, n_words, on_value = 1, off_value = None, axis = 1) 
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logit_words, labels=one_hot_labels)
        cross_ent_list.append(cross_entropy)
        
        
        #current_loss = tf.reduce_sum(cross_entropy)/batch_size
        #loss = loss + current_loss

    cross_entropy_tensor = tf.stack(cross_ent_list, 1)
    loss = tf.reduce_sum(cross_entropy_tensor, axis=1)
    loss = tf.divide(loss, tf.cast(sizeof_sentence, tf.float32))

    loss = tf.reduce_mean(loss, axis=0)
    summary = tf.summary.scalar('training_loss', loss)

    params = tf.trainable_variables()
    #optimizer = tf.train.AdamOptimizer(learning_rate)#.minimize(loss_op)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.minimize(loss)

    #train_step = optimizer.minimize(loss)
    
#     gradients, variables = zip(*optimizer.compute_gradients(loss))
#     gradients, _ = tf.clip_by_global_norm(gradients, 5.0)
#     train_op = optimizer.apply_gradients(zip(gradients, params))
    
#     logits = tf.stack(logits, axis = 0)
#     logits = tf.reshape(logits, (sizeof_sentence, batch_size, n_words))
#     logits = tf.transpose(logits, [1, 0, 2])
#     preds = tf.argmax(logits,2)
#     correct_pred = tf.equal(tf.argmax(preds,1), tf.argmax(y_label,1))
#     accuracy = tf.reduce_mean(correct_pred)

    logits = tf.stack(logits,axis=0)
    logits = tf.transpose(logits, [1, 0, 2])
    output_preds = tf.argmax(logits,2)


Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [22]:
np.shape(loss)

TensorShape([])

In [23]:
# with tf.variable_scope("encoding") as encoding_scope:
#     lstm_enc = tf.contrib.rnn.BasicLSTMCell(n_hidden)
#     _, last_state = tf.nn.dynamic_rnn(lstm_enc, inputs=input_embed, dtype=tf.float32)

In [24]:
# with tf.variable_scope("decoding") as decoding_scope:
#     # TODO: create the decoder LSTMs, this is very similar to the above
#     # you will need to set initial_state=last_state from the encoder
#     lstm_dec = tf.contrib.rnn.BasicLSTMCell(n_hidden)
#     dec_outputs, _ = tf.nn.dynamic_rnn(lstm_dec,inputs=output_embed, dtype=tf.float32)

In [25]:
# #connect outputs to 
# logits = tf.contrib.layers.fully_connected(dec_outputs, num_outputs=len(index_token), activation_fn=None) 

# with tf.name_scope("optimization"):
#     # Loss function
#     loss = tf.contrib.seq2seq.sequence_loss(logits, targets, tf.ones([batch_size, sizeof_sentence]))
#     # Optimizer
#     optimizer = tf.train.RMSPropOptimizer(1e-3).minimize(loss)

In [26]:
# output_dec.get_shape().as_list()

In [27]:
# state_dec[0].get_shape().as_list()

In [28]:
# x_video.get_shape().as_list()

In [29]:
# from utilities import show_graph
# show_graph(tf.get_default_graph().as_graph_def())

In [30]:
# def RNN(x, weights1, biases1):
    
#     x = tf.unstack(x,no_of_frames,1)
    
#     lstm_encoder = tf.keras.layers.LSTM(n_hidden, return_state=True) #reuse=tf.AUTO_REUSE)
#     output_encoder,state_h,state_c = lstm_encoder(x) #,dtype=tf.float32)
#     encoder_states = [state_h,state_c]
    
#     decoder
    
#     return tf.matmul(output1[-1],weights1) + bias1

In [31]:
# np.shape(vid_batch[1])

In [32]:
# logits = RNN(x_video,weights1,bias1)
# prediction = tf.nn.softmax(logits)


# loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_label))


# optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
# train_op = optimizer.minimize(loss_op)

# # Evaluate model (with test logits, for dropout to be disabled)
# correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(y_label, 1))
# accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [33]:
# batch_y = np.array(intencode_batch[1])
# np.shape(batch_y)

In [34]:
# batch_x = np.array(vid_batch[0])
# print(np.shape(batch_x))
# batch_x = np.reshape(batch_x,[-1,sizeof_frame])
# np.shape(batch_x)

In [ ]:
run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)

gpu_config = tf.ConfigProto()

with tf.Session(graph=graph,config=gpu_config) as sess:

    loss_list_train = []
    #loss_list_test
    preds_dict = {}

    sess.run(tf.global_variables_initializer())
    epochs = 10

    #training

    for epoch in range(epochs):

        for i in range(29):

            batch_x = np.array(vid_batch[i])
            #batch_x = np.reshape(batch_x,[-1,sizeof_frame])
            batch_y = np.array(intencode_batch[i])

            _, batch_loss, batch_preds = sess.run([train_op, loss,output_preds], feed_dict = {x_video: batch_x, y_label: batch_y})        

            loss_list_train.append(batch_loss)
            print(batch_loss)
            
       
    #testing
    
#     for i in range(29):

#         batch_x = np.array(vid_batch[i])
#         #batch_x = np.reshape(batch_x,[-1,sizeof_frame])
#         batch_y = np.array(intencode_batch[i])

#         _, batch_loss, batch_preds = sess.run([train_op, loss,output_preds], feed_dict = {x_video: batch_x, y_label: batch_y})        

#         loss_list_train.append(batch_loss)
#         print(batch_loss)
    
            #preds_dict[i] = batch_preds 

In [25]:
vid_batch[0]


[[array([0.        , 0.69352531, 0.        , ..., 0.        , 0.        ,
         0.        ]),
  array([0.        , 0.50499249, 0.        , ..., 0.        , 0.        ,
         0.        ]),
  array([0.        , 1.70716238, 0.        , ..., 0.        , 0.        ,
         0.        ]),
  array([0.       , 0.2650618, 0.       , ..., 0.       , 0.       ,
         0.       ]),
  array([0.        , 1.65669584, 0.        , ..., 0.        , 0.        ,
         0.        ]),
  array([0.        , 0.93898082, 0.        , ..., 0.        , 0.        ,
         0.        ]),
  array([0.        , 0.84765482, 0.        , ..., 0.        , 0.        ,
         0.        ]),
  array([0.07175767, 0.7388823 , 0.        , ..., 0.        , 0.28413364,
         0.        ]),
  array([0.       , 0.7665062, 0.       , ..., 0.       , 0.       ,
         0.       ]),
  array([0.        , 0.44792151, 0.        , ..., 0.        , 0.        ,
         0.        ]),
  array([0.        , 1.07996261, 0.       

In [ ]:
# batch_x = np.array(vid_batch[0])
# batch_y = np.array(intencode_batch[0])

# with tf.Session() as sess:
#     sess.run(init)

#     sess.run(train_op, feed_dict={x_video: batch_x, y_label: batch_y})

In [ ]:


# image_emb = tf.nn.xw_plus_b(x_video, weights1, bias1) 
# #image_emb = tf.reshape(image_emb, [batch_size, no_of_frames, n_hidden])

# #lstm2 = tf.keras.layers.LSTMCell(n_hidden)

# padding = tf.zeros([batch_size, n_hidden])


# #Only read the frames


        

            
                
# logit_words = tf.nn.xw_plus_b(output2, weights2, bias2)
# cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logit_words,onehot_encoded)

# loss = tf.reduce_sum(cross_entropy)

In [ ]:
# with tf.Session() as sess:
#     with sess.as_default():
#         print(tf.nn.embedding_lookup(onehot_encoded,[1]).eval())

In [ ]:
# inputs